[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/model-providers/databricks/rag_databricks.ipynb)

# Dependencies

Check that the weaviate-client and spark connector is installed:

In [0]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.8.1
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages
Requires: authlib, grpcio, grpcio-health-checking, grpcio-tools, httpx, pydantic, requests, validators
Required-by: 


In [0]:
%sql
LIST JAR io_weaviate_spark_connector_2_12_1_3_3.jar

Results
spark://10.213.208.171:43103/jars/io_weaviate_spark_connector_2_12_1_3_3.jar


# Imports

In [0]:
import weaviate
import weaviate.classes.config as wvcc
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import StringType, BooleanType
from weaviate.util import generate_uuid5
import time
from pyspark.sql import Row
from py4j.protocol import Py4JJavaError

# Weaviate Cloud Setup

Grab some secrets:

In [0]:
SCOPE="spark-connector-demo"

WEAVIATE_URL = dbutils.secrets.get(scope=SCOPE, key="WEAVIATE_URL")
WEAVIATE_API_KEY = dbutils.secrets.get(scope=SCOPE, key="WEAVIATE_API_KEY")
DATABRICKS_TOKEN = dbutils.secrets.get(scope=SCOPE, key="DATABRICKS_TOKEN")
EMBEDDINGS_ENDPOINT = dbutils.secrets.get(scope=SCOPE, key="EMBEDDINGS_ENDPOINT")
LLM_ENDPOINT = dbutils.secrets.get(scope=SCOPE, key="LLM_ENDPOINT")

Connect to weaviate and clear all collections:

In [0]:
auth_config = weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY)

client = weaviate.connect_to_wcs(cluster_url=WEAVIATE_URL,
                                 auth_credentials=auth_config,
                                 headers={
                                 "X-Databricks-Token": DATABRICKS_TOKEN
                                }
                            )

assert client.is_ready()

client.collections.delete_all()

<frozen importlib._bootstrap>:1049: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


In [0]:
print(f"Weaviate server version: {client.get_meta()['version']}")

Weaviate server version: 1.26.4


# Dataset

We'll ingest a job posting dataset stored in our catalog into Weaviate:

In [0]:
table_name = "coresignal_sample_coresignal_job_postings_data_largest_professional_network_indeed_jobs_3_other.coresignal_c4529d69_b6fb_4a7d_a444_a4ee04af0caf_samples.job_b190b4d4502744eda71e033975a0d136"

df = spark.table(table_name).select("title", "description").limit(10)
display(df)

title description DELIVERY TECHNICIAN Our Company\n\nAmerita\n\nOverview\n\nThe Delivery Technician is responsible for delivery of medications and supplies to Amerita’s home infusion patient’s residents. The delivery technicians are critical customer experience ambassadors supporting the needs of the patients by providing timely delivery of medication and supplies.\n\nResponsibilities\n\n• Plans and completes an efficient delivery or patient’s medication and supplies \n• Loads delivery vehicle with patient’s medication and supplies scheduled for delivery \n• Delivers patient’s medication and supplies to correct addresses, obtaining signatures and proof of receipt when necessary \n• Picks up equipment along the scheduled route, including items to be returned, company orders, and other items as requested \n• Maintains basic records of truck mileage and deliveries made \n• Ensures that delivery vehicle receives basic service according to maintenance schedule; reports larger service needs and mechanical issues to supervisor \n• Maintains and clean and organized delivery vehicle in accordance with ACHC accreditations standards \n• Assists in general warehouse duties when delivery route is completed, as needed and eligible per branch warehouse coordinators criteria \n• Performs other related duties as assigned\n\n\nQualifications\n\n• High School Diploma/GED or equivalent, preferred \n• Excellent customer service orientation \n• Ability to drive safely \n• Ability to use navigation tools such as onboard GPS, smartphone apps, and/or web-based or physical maps \n• Ability to plan delivery routes \n• Ability to maintain basic logs and records \n• Ability to carry out a series of instructions without constant supervision \n• Ability to remain professional and courteous at all times\n\n\nAbout Our Line Of Business\n\nAmerita is a specialty infusion company focused on providing complex pharmaceutical products and clinical services to patients outside of the hospital. Committed to excellent service, our vision is to combine the administrative efficiencies of a large organization with the flexibility, responsiveness and entrepreneurial spirit of a local provider. For more information about Amerita, please visit www.ameritaiv.com . Follow us on Twitter and professionalnetwork .\n\n Salary Range\n\nUSD $12.00 - $15.00 / Hour Qualified Gas Engineer - Slough Qualified Gas Engineer\n\nUxbridge Road, Slough, Berkshire, SL2 5NA\n\nBasic salary up to £43,430* per annum\n\n• Plus earn more through standby and call-out allowances and performance bonuses. \n• Conditions apply\n\n\n\nLet’s fuel your career and help you thrive.\n\nAbout The Role\n\nAs part of our Emergency Response Team, you’ll be the first on the scene at customers’ homes, which makes you the face of our brand for people who are worried about potential gas escapes or carbon monoxide activated alarms.\n\nSometimes you’ll be dealing with the emergency services on behalf of our customers, making you part of a team that’s responsible for protecting people’s lives. They could be vulnerable, elderly, or simply looking for help, and it’s that level of empathy, plus the pride you’ll feel from caring for people and keeping them safe in their homes that could make you a Cadent person.\n\nIt’s a role where no two days are the same, so from installation, replacement, and maintenance of gas meters for our home and business customers, to high rise and gas safety regulation surveys, to fitting cooker safety valves, when we say it’s a varied role, we mean it.\n\nSo, what’s in it for you?\n\n• 33 days holiday, inclusive of bank holidays\n• A company van (Work purpose use only)\n• Subsidised Gympass membership\n• Pension Scheme – with company contributions double matched up to 12%\n• Retail and Leisure discounts \n• Flexible Benefits, including private medical, cycle2work scheme, car lease, gadget discounts, plus much more!\n• Annual performance bonus\n• Unlimited access to our Occupational Health and E

Add a uuid to make sure each row is unique:

In [0]:
def create_uuid(title, description):
    obj = {
        "title": title,
        "description": description
    }
    return str(generate_uuid5(obj))

create_uuid_udf = udf(create_uuid, StringType())

df = df.withColumn("uuid", create_uuid_udf(df["title"], df["description"]))\
        .select("uuid", "title", "description")

display(df)

uuid title description 52fad351-0bef-5550-8af5-3d1f89a3fcf8 DELIVERY TECHNICIAN Our Company\n\nAmerita\n\nOverview\n\nThe Delivery Technician is responsible for delivery of medications and supplies to Amerita’s home infusion patient’s residents. The delivery technicians are critical customer experience ambassadors supporting the needs of the patients by providing timely delivery of medication and supplies.\n\nResponsibilities\n\n• Plans and completes an efficient delivery or patient’s medication and supplies \n• Loads delivery vehicle with patient’s medication and supplies scheduled for delivery \n• Delivers patient’s medication and supplies to correct addresses, obtaining signatures and proof of receipt when necessary \n• Picks up equipment along the scheduled route, including items to be returned, company orders, and other items as requested \n• Maintains basic records of truck mileage and deliveries made \n• Ensures that delivery vehicle receives basic service according to maintenance schedule; reports larger service needs and mechanical issues to supervisor \n• Maintains and clean and organized delivery vehicle in accordance with ACHC accreditations standards \n• Assists in general warehouse duties when delivery route is completed, as needed and eligible per branch warehouse coordinators criteria \n• Performs other related duties as assigned\n\n\nQualifications\n\n• High School Diploma/GED or equivalent, preferred \n• Excellent customer service orientation \n• Ability to drive safely \n• Ability to use navigation tools such as onboard GPS, smartphone apps, and/or web-based or physical maps \n• Ability to plan delivery routes \n• Ability to maintain basic logs and records \n• Ability to carry out a series of instructions without constant supervision \n• Ability to remain professional and courteous at all times\n\n\nAbout Our Line Of Business\n\nAmerita is a specialty infusion company focused on providing complex pharmaceutical products and clinical services to patients outside of the hospital. Committed to excellent service, our vision is to combine the administrative efficiencies of a large organization with the flexibility, responsiveness and entrepreneurial spirit of a local provider. For more information about Amerita, please visit www.ameritaiv.com . Follow us on Twitter and professionalnetwork .\n\n Salary Range\n\nUSD $12.00 - $15.00 / Hour 55c15892-65b1-571b-b107-a472b5ba8553 Qualified Gas Engineer - Slough Qualified Gas Engineer\n\nUxbridge Road, Slough, Berkshire, SL2 5NA\n\nBasic salary up to £43,430* per annum\n\n• Plus earn more through standby and call-out allowances and performance bonuses. \n• Conditions apply\n\n\n\nLet’s fuel your career and help you thrive.\n\nAbout The Role\n\nAs part of our Emergency Response Team, you’ll be the first on the scene at customers’ homes, which makes you the face of our brand for people who are worried about potential gas escapes or carbon monoxide activated alarms.\n\nSometimes you’ll be dealing with the emergency services on behalf of our customers, making you part of a team that’s responsible for protecting people’s lives. They could be vulnerable, elderly, or simply looking for help, and it’s that level of empathy, plus the pride you’ll feel from caring for people and keeping them safe in their homes that could make you a Cadent person.\n\nIt’s a role where no two days are the same, so from installation, replacement, and maintenance of gas meters for our home and business customers, to high rise and gas safety regulation surveys, to fitting cooker safety valves, when we say it’s a varied role, we mean it.\n\nSo, what’s in it for you?\n\n• 33 days holiday, inclusive of bank holidays\n• A company van (Work purpose use only)\n• Subsidised Gympass membership\n• Pension Scheme – with company contributions double matched up to 12%\n• Retail and Leisure discounts \n• Flexible Benefits, including private medical, cycle2work scheme, car lease, gadget discounts, plus much more!\n

# Ingest

Create the collection in Weaviate:

In [0]:
if client.collections.exists("JobAds"):
    client.collections.delete("JobAds")

collection = client.collections.create(
    name="JobAds",
    vectorizer_config=[
        wvcc.Configure.NamedVectors.text2vec_databricks(
            name="default", 
            endpoint=EMBEDDINGS_ENDPOINT
        )
    ],
    generative_config=wvcc.Configure.Generative.databricks(
        endpoint=LLM_ENDPOINT, 
        max_tokens=8092
    ),
    properties=[
        wvcc.Property(
            name="title",
            data_type=wvcc.DataType.TEXT
        ),
        wvcc.Property(
            name="description",
            data_type=wvcc.DataType.TEXT
        )
    ]
)
    
collection = client.collections.get("JobAds")

assert collection.aggregate.over_all(total_count=True).total_count == 0

Write the dataframe to Weaviate:

In [0]:
df.write.format("io.weaviate.spark.Weaviate") \
    .option("batchSize", 200) \
    .option("scheme", "https") \
    .option("host", WEAVIATE_URL.replace("https://","")) \
    .option("apiKey", WEAVIATE_API_KEY) \
    .option("header:X-Databricks-Token", DATABRICKS_TOKEN) \
    .option("className", "JobAds") \
    .option("id", "uuid") \
    .mode("append") \
    .save()


# Sanity Checks

Check that the collection has 10 objects:

In [0]:
total_count_in_weaviate = collection.aggregate.over_all(total_count=True).total_count
assert total_count_in_weaviate == 10, f"Expected 10, got {total_count_in_weaviate}"

Check that the object has a vector:

In [0]:
retry_attempts = 3
for attempt in range(retry_attempts):
    try:
        obj = collection.query.fetch_objects(limit=1, include_vector=True).objects[0]
        break
    except Exception as e:
        if attempt < retry_attempts - 1:
            time.sleep(2)  # wait before retrying
        else:
            raise e
vector = obj.vector['default']

assert vector
assert len(vector) == 1024

Randomly sample the dataframe and check each row is identifcal in weaviate:

In [0]:
# Define the UDF
@udf(returnType=BooleanType())
def check_weaviate(uuid, title, description):
    try:
        # Establish a connection to Weaviate
        client = weaviate.connect_to_wcs(cluster_url=WEAVIATE_URL,
                                          auth_credentials=auth_config)
        collection = client.collections.get("JobAds")
        
        # Fetch the object by UUID
        obj = collection.query.fetch_object_by_id(uuid)
        
        # Check each obj's properties
        if obj is not None:
            return obj.properties['title'] == title and obj.properties['description'] == description
        
        return False
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return False

num_not_match = df.sample(0.5) \
    .withColumn("is_match", 
                check_weaviate(col("uuid"), col("title"), col("description"))) \
    .filter(col("is_match") == False) \
    .count()

assert num_not_match == 0



# Search

Use hybrid search to search for a job:

In [0]:
response = collection.query.hybrid(
    query="Wash clothes",
    query_properties=["title"],
    limit=3
)

rows = [Row(uuid=str(obj.uuid), title=obj.properties['title'], description=obj.properties['description']) for obj in response.objects]

df_response = spark.createDataFrame(rows)

display(df_response)

uuid title description 6e12ee46-621c-554c-af9a-e540e185f798 Laundry Worker Who We Are\n\nHealthcare Services Group (HCSG) is an experienced partner managing housekeeping, laundry, dining, and nutritional services within the healthcare market. For over 45 years, we have provided essential services to thousands of healthcare communities across the country. As one of America's Most Trustworthy Companies, we have been recognized for treating our customers and employees fairly and pursuing excellence via an ever-evolving and expanding focus on training and the development of team members at every level.\n\nOur Company Purpose & Values\n\nOur Purpose is Fostering Fulfillment In Communities.\n\nOur Values, Integrity, Collaboration, Empowerment, Passion & Perseverance (aka Grit) influence our decisions and the actions that follow in and out of the workplace.\n\nWho You Are\n\nYou are driven, ambitious, have a personal calling for serving people, and desire to positively impact your community. You enjoy collaborating with others, aren't afraid to roll up your sleeves, and go beyond to get the job done. You have a passion and desire to deliver the highest customer and resident experience every day.\n\nWhat We Offer\n\nHCSG offers a comprehensive array of benefits to support our associates and their family’s health and well-being. Our goal is to promote welfare and health and enhance our associates' and their families' overall quality of life.\n\n• Benefits - FREE Telemedicine and Prescription Discount Program along with an array of medical, dental, vision, and other supplemental health plans. *Availability based on state.\n• Training - We take the development and education of our associates very seriously. Our promotions-from-within philosophy opens the door to a stimulating and robust career with HCSG!\n• Employee Assistance Programs - FREE access for employees and their families for health, financial, and family services to support our employees' welfare and mental and physical health. \n• Employee Recognition Programs - We strive to recognize employees who live out our Company Purpose and are committed to #GoingBeyond for our Client, Resident, and Employee Communities!\n• Going Beyond Assistance Fund - A 501(c)(3) nonprofit formed by HCSG to serve as a charitable employer-sponsored disaster relief organization to provide emergency, hardship, and disaster assistance, to HCSG employees and their dependents or family members.\n• Paid - Holidays and Vacation for eligible employees.\n• Employee Stock Purchase Plan - Investment opportunity available to all eligible employees after two years of service.\n• Nationwide Opportunities - Transfers are available Nationwide – great for active Military and Family!\n\n\n\nKeeping You Safe\n\n• The safety and health of our employees and that of the resident communities that we serve are our Company's most significant priority. \n• Personal Protective Equipment - All employees will be provided and required to wear appropriate PPE for all functions. \n• COVID-19 Vaccination - Employees will be required to comply with all policies, as required by State, Local, and/or our Customers\n\n\n\nWhy HCSG\n\nAt HCSG, you will find an environment that is fulfilling, collaborative, committed, stimulating, and structured to help you succeed in serving our communities. We offer engaging opportunities for every phase of your career. HCSG is committed to providing growth and leadership opportunities to see you succeed!\n\nPosition Summary\n\n• Laundry Worker - Responsible for collecting, sorting, and loading soiled linen into the washer(s) and loading clean linens into the dryer(s), folding clean linens, counting and recording the linen processed each day, making proper deliveries to units and, cleaning and sanitizing the work area including; machines, work tables, and sorting area.\n• Safety - Utilize protective gear in all appropriate functions. Responsible for the safe and proper mixing and use of cleaning solution

# Generative Search

Ask the LLM to translate each job:

In [0]:
generate_prompt = "Produce a markdown table of the job title and its translation in german"

response = collection.generate.near_text(
    query="Wash clothes",
    return_properties=["title"],
    grouped_task=generate_prompt,
    limit=3
)

print(response.generated)

| Job Title | German Translation |
|---|---|
| Laundry Worker | Wäschereiarbeiter |
| Wire Drawer Technician | Drahtzieh-Techniker |
| Resident Care Companion or STNA - DSL Elyria | Pflegebegleiter oder STNA - DSL Elyria |


# Upsert

Change a field in a row and upload it to weaviate:

In [0]:
# pick a uuid
sample_uuid = df.first().uuid

# show the row in df
print(f"original row:")
df.filter(col("uuid") == sample_uuid).show()

# update the job title
print(f"updated row:")
updated_df = df.withColumn("title", when(col("uuid") == sample_uuid, "foo").otherwise(col("title"))).filter(col("uuid") == sample_uuid)

updated_df.show()

updated_df.write.format("io.weaviate.spark.Weaviate") \
    .option("batchSize", 200) \
    .option("scheme", "https") \
    .option("host", WEAVIATE_URL.replace("https://","")) \
    .option("apiKey", WEAVIATE_API_KEY) \
    .option("header:X-Databricks-Token", DATABRICKS_TOKEN) \
    .option("className", "JobAds") \
    .option("id", "uuid") \
    .mode("append")\
    .save()

# get the updated object from weaviate
updated_obj = collection.query.fetch_object_by_id(sample_uuid)
print("updated object in weaviate:")
print(f"uuid: {updated_obj.uuid}, title: {updated_obj.properties['title']}")


<frozen importlib._bootstrap>:1049: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
/usr/lib/python3.11/socket.py:776: ResourceWarning: unclosed <socket.socket fd=103, family=2, type=1, proto=6, laddr=('127.0.0.1', 54442), raddr=('127.0.0.1', 36897)>
  self._sock = None


original row:
+--------------------+-------------------+--------------------+
|                uuid|              title|         description|
+--------------------+-------------------+--------------------+
|52fad351-0bef-555...|DELIVERY TECHNICIAN|Our Company\n\nAm...|
+--------------------+-------------------+--------------------+

updated row:
+--------------------+-----+--------------------+
|                uuid|title|         description|
+--------------------+-----+--------------------+
|52fad351-0bef-555...|  foo|Our Company\n\nAm...|
+--------------------+-----+--------------------+

updated object in weaviate:
uuid: 52fad351-0bef-5550-8af5-3d1f89a3fcf8, title: foo


# Error Handling

Make sure the collection exists in weaviate before writing to it:

In [0]:
try:
    df.write.format("io.weaviate.spark.Weaviate") \
        .option("scheme", "https") \
        .option("host", WEAVIATE_URL.replace("https://", "")) \
        .option("apiKey", WEAVIATE_API_KEY) \
        .option("className", "JobAds2") \
        .mode("append").save()
except Py4JJavaError as e:
    if 'WeaviateClassNotFoundError' in str(e.java_exception):
        print("WeaviateClassNotFoundError: The specified class 'JobAds2' was not found.")

WeaviateClassNotFoundError: The specified class 'JobAds2' was not found.


<frozen importlib._bootstrap>:1049: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


The dataframe must contain all the properties defined by the collection in weaviate, otherwise it will throw a [CANNOT_FIND_DATA](https://docs.databricks.com/en/error-messages/incompatible-data-for-table-error-class.html#cannot_find_data) error:

In [0]:
# df.drop("title").write.format("io.weaviate.spark.Weaviate") \
#     .option("batchSize", 200) \
#     .option("scheme", "https") \
#     .option("host", WEAVIATE_URL.replace("https://","")) \
#     .option("apiKey", WEAVIATE_API_KEY) \
#     .option("header:X-Openai-Api-Key", OPENAI_API_KEY) \
#     .option("className", "JobAds") \
#     .option("id", "uuid") \
#     .mode("append").save()

# this will throw:
# [INCOMPATIBLE_DATA_FOR_TABLE.CANNOT_FIND_DATA] Cannot write incompatible data for the table `JobAds`: Cannot find data for the output column `title`. SQLSTATE: KD000


If disk usage of a node in the Weaviate cluster is higher than the `DISK_USE_READONLY_PERCENTAGE` percentage, then all shards on the affected node will be marked as `READONLY`, meaning all future write requests will fail.

To recover from this, increase the node's disk capacity and mark the shards as ready again. Refer to the weaviate [docs](https://weaviate.io/developers/weaviate/configuration/persistence#disk-pressure-warnings-and-limits) for detailed steps

# Further Reading


* Databricks [integration docs](https://weaviate.io/developers/integrations/data-platforms/databricks)